# LangChain을 활용한 간단한 Q&A Applicaion

### RAG흐름
**Pre-processing**
1. 문서로드(Document Load): 문서 내용을 불러옵니다.
2. 분할(Text Split): 문서를 특정 기준(Chunk) 으로 분할합니다.
3. 임베딩(Embedding): 분할된(Chunk) 를 임베딩하여 저장합니다.
4. 벡터DB 저장: 임베딩된 Chunk 를 DB에 저장합니다.

**RunTime**
1. 검색기(Retriever): 쿼리(Query) 를 바탕으로 DB에서 검색하여 결과를 가져오기 위하여 리트리버를 정의합니다. 리트리버는 검색 알고리즘이며(Dense, Sparse) 리트리버로 나뉘게 됩니다.
    - Dense: 유사도 기반 검색(FAISS, DPR)
    - Sparse: 키워드 기반 검색(BM25, TF-IDF)
2. 프롬프트: RAG 를 수행하기 위한 프롬프트를 생성합니다. 프롬프트의 context 에는 문서에서 검색된 내용이 입력됩니다. 프롬프트 엔지니어링을 통하여 답변의 형식을 지정할 수 있습니다.
3. LLM: 모델을 정의합니다.(GPT-3.5, GPT-4, Claude, etc..)
4. Chain: 프롬프트 - LLM - 출력 에 이르는 체인을 생성합니다.

## 1. LOAD

In [25]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [10]:
import os
os.environ["LANGCHAIN_PROJECT"] = "RAG TUTORIAL"

In [17]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("RAG TUTORIAL")

LangSmith 추적을 시작합니다.
[프로젝트명]
RAG TUTORIAL


In [16]:
import bs4 #웹 페이지 파싱을 위한 라이브러리
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter #텍스트 분할
from langchain_community.document_loaders import WebBaseLoader #문서로딩
from langchain_community.vectorstores import FAISS #벡터 저장
from langchain_core.output_parsers import StrOutputParser #출력 파싱
from langchain_core.runnables import RunnablePassthrough 
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


### 1. Load

In [19]:
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000378416",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

문서의 수: 1


[Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content="\n출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책\n\n\n[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다.[오현석/부영그룹 직원 : 아이 키우는 데 금전적으로 많이 힘든 세상이잖아요. 교육이나 생활하는 데 큰 도움이 될 거라 생각합니다.]만약 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.[이중근/부영그룹 회장 : 3년 이내에 세 아이를 갖는 분이 나올 것이고 따라서 주택을 제공할 수 있는 계기가 될 것으로 생각하고.][조용현/부영그룹 직원 : 와이프가 셋째도 갖고 싶어 했는데 경제적 부담 때문에 부정적이었거든요. (이제) 긍정적으로 생각할 수 있을 것 같습니다.]오늘 행사에서는, 회사가 제공하는 출산장려금은 받는 

### 2. Split

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, #1000글자 단위로 쪼갬
                                               chunk_overlap=100) #쪼개진 부분이 자연스럽도록 overlap 설정

splits = text_splitter.split_documents(docs)
len(splits) #3개의 문서로 쪼개짐

3

### 3. Embedding, Vector DB 저장

In [28]:
# 벡터스토어를 생성합니다. (DB에 저장)
vectorstore = FAISS.from_documents(documents=splits, 
                                   embedding=OpenAIEmbeddings()) #1536차원 

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.(검색을 위한 retriever 생성)
retriever = vectorstore.as_retriever()
#연관성이 높은 부분을 가져옴

In [30]:
#문자열 파싱
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

# 아래와 같이 프롬프트 다운받아 입력 가능
# prompt = hub.pull("teddynote/rag-prompt-korean")
# prompt

In [31]:
llm = ChatOpenAI(model_name = "gpt-4-turbo-preview", 
                temperature=0)

def format_docs(docs):
    #검색한 문서 결과를 하나의 문단으로 합쳐줌
    return "\n\n".join(doc.page_content for doc in docs)
    #Split중 n개의 문서를 가져올 수 있는데, 여러 문서를 하나의 문서로 합쳐주는 함수

# 체인을 생성합니다.
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt 
    | llm
    | StrOutputParser()
)
#RunnablePassthrough()로 질문이 들어감 
#질문기반으로 retriever 즉 검색을 수행(유사도가 높은 단락을 뽑아냄)
#문장을 prompt로 넘겨줌
#llm이 처리

In [32]:
rag_chain.invoke(
    "부영그룹의 출상 장려 정책이 뭐야?"
)

'부영그룹의 출산 장려 정책은 2021년 이후 태어난 직원 자녀에게 1억원씩, 총 70억원을 지원하는 것입니다. 해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 되며, 셋째까지 낳는 경우에는 국민주택을 제공하겠다는 정책도 포함되어 있습니다.'

In [37]:
rag_chain.invoke("부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?")


'부영그룹은 출산 직원에게 1억원의 지원을 제공합니다.'

In [39]:
rag_chain.invoke("부영그룹의 임직원 숫자는 몇명인가요?")

'주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다'